In [1]:
import requests
from bs4 import BeautifulSoup
import re
from pandas import DataFrame
import pandas as pd
import time
from datetime import datetime, timedelta

In [2]:
href_list=[]
title_list=[]
press_list = []
date_list = []
content_list = []

In [3]:
#html수집기-기본1 : keyword만
def searchNewsExtractor1(keyword): #수동
    global soup
    pageNo= 4001
    for i in range(5): #809
        pageNo+=10
        url = "https://search.naver.com/search.naver?&where=news&query="+keyword+"&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:da,p:all,a:all&mynews=0&start="+str(pageNo)+"&refresh_start=0"
        req = requests.get(url)
        html = req.text
        soup = BeautifulSoup(html, 'lxml')
        print(pageNo,"페이지 상황: ",req.status_code)

        newsParser()

In [4]:
#html수집기-심화2: keyword/ 정렬순서(최신/과거)/ 날짜 지정 
def searchNewsExtractor2(keyword): #자동
    global soup
    
    for i in range(400): #4,000건이 최대
        pageNo= str(10*i+1)
        #오래된순
        #url = "https://search.naver.com/search.naver?&where=news&query="+keyword+"&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:da,p:all,a:all&mynews=0&start="+pageNo+"&refresh_start=0"
        #최신순
        #url = "https://search.naver.com/search.naver?&where=news&query="+keyword+"&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:dd,p:all,a:all&mynews=0&start="+pageNo+"&refresh_start=0"
        
        #특정기간( 시작날짜 ~ 종료날짜)/ 오래된순
        url = "https://search.naver.com/search.naver?&where=news&query="+keyword+"&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start="+str(pageNo)+"&refresh_start=0"
        req = requests.get(url)
        
        html = req.text
        soup = BeautifulSoup(html, 'lxml')
        
        print(url)
        print(i,"페이지 상황: ",req.status_code)

        newsParser()

In [66]:
sourceNdate= soup.find_all("dd",{"class":"txt_inline"})
for i in sourceNdate:
    print(type(i.text),i.text)

<class 'str'> 매일일보  6일 전   보내기  
<class 'str'> 중앙일보언론사 선정  6일 전  네이버뉴스   보내기  
<class 'str'> 테크M  6일 전   보내기  
<class 'str'> IT조선  6일 전   보내기  
<class 'str'> 위키트리  7일 전   보내기  
<class 'str'> 한국경제  A17면1단  7일 전  네이버뉴스   보내기  
<class 'str'> 전자신문  7면2단  7일 전  네이버뉴스   보내기  
<class 'str'> EBN  7일 전   보내기  
<class 'str'> 미디어펜  7일 전   보내기  
<class 'str'> 한국경제언론사 선정  7일 전  네이버뉴스   보내기  


In [49]:
dateli= ["매일일보  6일 전   보내기 ", "한국경제언론사 선정  7시간 전  네이버뉴스   보내기"]
date_l =[]

for i in dateli:
    if "일" in i:
        match = re.search('\d+일 전', i)
        date_l.append(str(match.group())) 
    elif "시간" in i:
        match = re.search('\d+시간 전', i)
        date_l.append(str(match.group()))
    else: pass # date_l.append(i.text)

print(date_l)

['6일 전', '7시간 전']


In [5]:
def newsParser():
    global href_list
    global title_list
    global press_list
    global date_list
    global content_list

    #1. 링크, 2. 타이틀
    hrefNtitle = soup.find_all(class_=[" _sp_each_title","_sp_each_url _sp_each_title"])    
    for i in hrefNtitle:
        href_list.append(i['href']) 
        title_list.append(i['title'])
        
    #3. 언론사
    press = soup.find_all("span",{"class":"_sp_each_source"})
    for i in press:
        press_list.append(i.text)

    #4. 날짜(3가지 유형 - 날짜/일 전/ 시간 전)
    sourceNdate= soup.find_all("dd",{"class":"txt_inline"})

#     #오래전 뉴스 수집 시
#     for i in sourceNdate:
#         match = re.search(r'\d{4}.\d{2}.\d{2}', i.text)
#         date_list.append(str(match.group()))    
    
    #최근 뉴스 수집 시
    for i in sourceNdate:
        try: 
            match = re.search(r'\d{4}.\d{2}.\d{2}', i.text)
            date_list.append(str(match.group()))
        except AttributeError:
            try: 
                if "일" in i.text:
                    match = re.search('\d+일 전', i.text)
                    date_list.append(str(match.group())) 
                elif "시간" in i.text:
                    match = re.search('\d+시간 전', i.text)
                    date_list.append(str(match.group()))
                else: pass
            except AttributeError: 
                date_list.append(i.text)

            
    # 5. 본문
    articles = soup.select('.type01 > li')
    for i in range(10):
        content_list.append(articles[i].select('dl > dd')[1].text)

In [6]:
def collect2Excel():
    global df1
    df1 =  DataFrame({"press": press_list, 
                      "title": title_list, 
                      "date":date_list, 
                      "content": content_list,
                      "url":href_list})
    return df1

In [94]:
#<<<수동버전>>>>
keyword = "카운터포인트리서치"
searchNewsExtractor1(keyword)
collect2Excel()

4011 페이지 상황:  200
4021 페이지 상황:  200
4031 페이지 상황:  200
4041 페이지 상황:  200
4051 페이지 상황:  200


,press,title,date,content,url
0,이코노믹리뷰,"갤럭시S9, 올해 2분기 스마트폰 시장 ‘톱’",2018.08.31,시장조사업체 카운터포인트리서치는 31일 국내 2분기 스마트폰 시장에서 삼성전자의 갤...,http://www.econovill.com/news/articleView.html...
1,녹색경제신문,"2분기 국내서 가장 많이 팔린 폰은 '갤S9'...아이폰X는 3위, LG전자는 10...",2018.08.31,31일 글로벌 시장조사업체 카운터포인트리서치에 따르면 올 해 2분기 국내에서 가장 ...,http://www.greened.kr/news/articleView.html?id...
2,아이티데일리,2분기 국내 최다 판매 스마트폰은 '삼성 갤럭시S9',2018.09.01,카운터포인트리서치는 올 2분기 국내 스마트폰 판매량 조사결과를 31일 발표했다. 삼...,http://www.itdaily.kr/news/articleView.html?id...
3,이데일리언론사 선정,"삼성·애플 스마트폰, 2분기 판매 10위권 독식..갤S9 1위",2018.09.01,1일 글로벌 시장조사업체 카운터포인트리서치는 올 2분기 삼성 갤럭시S9이 전체 판매...,http://www.edaily.co.kr/news/newspath.asp?news...
4,서울경제,2Q 국내 스마트폰 시장 ‘삼성 천하’...점유율 48%,2018.09.01,1일 글로벌 시장조사업체 카운터포인트리서치에 따르면 2·4분기 국내 스마트폰 상위 ...,http://www.sedaily.com/NewsView/1S4GTRNHCN
...,...,...,...,...,...
95,머니S,"갤럭시S9, 올 2분기 한국서 가장 많이 팔렸다",2018.09.01,1일 시장조사업체 카운터포인트리서치에 따르면 갤럭시S9은 2분기 국내 스마트폰 판매...,http://moneys.mt.co.kr/news/mwView.php?no=2018...
96,뉴시스,'짝퉁서 프리미엄폰까지' 中휴대폰 천차만별,2018.09.02,2018.08.23 odong85@newsis.com 카운터포인트리서치 조사에 따르...,http://www.newsis.com/view/?id=NISX20180831_00...
97,뉴스핌,[IFA 2018] 최신 모바일 드문 LG전자 전시관,2018.09.02,반면 올해 2분기 세계 스마트폰 시장 2위(카운터포인트리서치 기준)에 오른 화웨이는...,http://www.newspim.com/news/view/20180901000064
98,케이벤치,"'갤럭시S9' 시리즈, 2분기 국내 '베스트셀링폰'.. 아이폰X 3위",2018.09.03,지난 1일 시장조사업체 카운터포인트리서치가 발표한 보고서에 따르면 갤럭시S9과 갤럭...,http://kbench.com/?q=node/190899


In [133]:
#<<<자동버전>>>>
#카운터포인트리서치/ 스트래티지애널리틱스
keyword = "스트래티지애널리틱스"

searchNewsExtractor2(keyword)
collect2Excel()

https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=1&refresh_start=0
0 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=11&refresh_start=0
1 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=21&refresh_start=0
2 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=31&refresh_start=0
3 페이지 상황:  200
https

https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=331&refresh_start=0
33 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=341&refresh_start=0
34 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=351&refresh_start=0
35 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=361&refresh_start=0
36 페이지 상황:  

https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=661&refresh_start=0
66 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=671&refresh_start=0
67 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=681&refresh_start=0
68 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=691&refresh_start=0
69 페이지 상황:  

https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=991&refresh_start=0
99 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=1001&refresh_start=0
100 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=1011&refresh_start=0
101 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=1021&refresh_start=0
102 페이지

https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=1321&refresh_start=0
132 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=1331&refresh_start=0
133 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=1341&refresh_start=0
134 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=1351&refresh_start=0
135 페

...

https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=2641&refresh_start=0
264 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=2651&refresh_start=0
265 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=2661&refresh_start=0
266 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=2671&refresh_start=0
267 페

https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=2971&refresh_start=0
297 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=2981&refresh_start=0
298 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=2991&refresh_start=0
299 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=3001&refresh_start=0
300 페

https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=3301&refresh_start=0
330 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=3311&refresh_start=0
331 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=3321&refresh_start=0
332 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=3331&refresh_start=0
333 페

https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=3631&refresh_start=0
363 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=3641&refresh_start=0
364 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=3651&refresh_start=0
365 페이지 상황:  200
https://search.naver.com/search.naver?&where=news&query=스트래티지애널리틱스&sm=tab_pge&sort=2&photo=0&field=0&reporter_article=&pd=3&ds=2018.11.15&de=2020.04.13&docid=&nso=so:dd,p:from20181115to20200413,a:all&mynews=0&start=3661&refresh_start=0
366 페

IndexError: list index out of range

In [134]:
collect2Excel()

,press,title,date,content,url
0,동아일보,갤S10에 7배 빠른 ‘모바일 두뇌’ 탑재,2018.11.15,될 것”이라고 카운터포인트리서치는 지난해 10월 분석했다. 한편 시장조사업체 스트래...,http://news.donga.com/3/all/20181114/92877192/1
1,뉴시스언론사 선정,AI스피커 올해 1억대 넘나? 中 알리바바·바이두 '맹추격',2018.11.15,15일 글로벌 시장조사업체 스트래티지 애널리틱스(SA)에 따르면 올해 3분기 글로벌...,http://www.newsis.com/view/?id=NISX20181115_00...
2,뉴스토마토,"LG전자 휴대폰, 글로벌 점유율 1%대로 하락",2018.11.15,5%에서 올 3분기 18%로 후퇴했다. 시장조사기관 스트래티지애널리틱스(SA)와 카...,http://www.newstomato.com/ReadNews.aspx?no=858336
3,ZDNet Korea,"AI스피커 각축…아마존, 다시 구글 제쳤다",2018.11.16,시장조사업체 스트래티지 애널리틱스는 지난 9월 구글 홈 미니가 2분기 음성 스피커 ...,http://www.zdnet.co.kr/ArticleView.asp?artice_...
4,뉴스핌,"'AI·5G' 카드 꺼낸 삼성, 내년 비메모리 지각변화 기대",2018.11.16,시장조사업체 스트래티지애널리틱스(SA)에 따르면 올해 삼성전자와 퀄컴의 모바일 프로...,http://www.newspim.com/news/view/20181116000437
...,...,...,...,...,...
3741,서울경제언론사 선정,"삼성전자, 코로나 위기서도 반도체 덕분에 버텼다",1일 전,IM(IT·모바일) 부문 또한 스트래티지애널리틱스(SA)가 올해 글로벌 스마트폰 판...,https://www.sedaily.com/NewsView/1Z1G2AWK84
3742,메트로신문,"최근 자율주행 배달·물류·순찰로봇 등 출시 잇따라, 자율주행 로봇 시장 경쟁 '후끈'",1일 전,시장조사기관 스트래티지 애널리틱스에 따르면 배송(배달)·물류 로봇 등 전문 서비스 ...,http://www.metroseoul.co.kr/article/2020041250...
3743,디지털데일리언론사 선정,"LG폰 브랜드 ‘통일→개성’ 전환…벨벳, 제2의 초콜릿 될까(종합)",1일 전,시장조사기관 스트래티지애널리틱스(SA)에 따르면 작년 LG전자 판매량 순위는 8위다...,http://www.ddaily.co.kr/news/article.html?no=1...
3744,디지털타임스,先극장 後온라인시대 막내리나,1일 전,시장조사업체 스트래티지 애널리틱스는 최근 넷플릭스를 포함한 글로벌 OTT 신규 가입...,http://www.dt.co.kr/contents.html?article_no=2...


In [135]:
df1

,press,title,date,content,url
0,동아일보,갤S10에 7배 빠른 ‘모바일 두뇌’ 탑재,2018.11.15,될 것”이라고 카운터포인트리서치는 지난해 10월 분석했다. 한편 시장조사업체 스트래...,http://news.donga.com/3/all/20181114/92877192/1
1,뉴시스언론사 선정,AI스피커 올해 1억대 넘나? 中 알리바바·바이두 '맹추격',2018.11.15,15일 글로벌 시장조사업체 스트래티지 애널리틱스(SA)에 따르면 올해 3분기 글로벌...,http://www.newsis.com/view/?id=NISX20181115_00...
2,뉴스토마토,"LG전자 휴대폰, 글로벌 점유율 1%대로 하락",2018.11.15,5%에서 올 3분기 18%로 후퇴했다. 시장조사기관 스트래티지애널리틱스(SA)와 카...,http://www.newstomato.com/ReadNews.aspx?no=858336
3,ZDNet Korea,"AI스피커 각축…아마존, 다시 구글 제쳤다",2018.11.16,시장조사업체 스트래티지 애널리틱스는 지난 9월 구글 홈 미니가 2분기 음성 스피커 ...,http://www.zdnet.co.kr/ArticleView.asp?artice_...
4,뉴스핌,"'AI·5G' 카드 꺼낸 삼성, 내년 비메모리 지각변화 기대",2018.11.16,시장조사업체 스트래티지애널리틱스(SA)에 따르면 올해 삼성전자와 퀄컴의 모바일 프로...,http://www.newspim.com/news/view/20181116000437
...,...,...,...,...,...
3741,서울경제언론사 선정,"삼성전자, 코로나 위기서도 반도체 덕분에 버텼다",1일 전,IM(IT·모바일) 부문 또한 스트래티지애널리틱스(SA)가 올해 글로벌 스마트폰 판...,https://www.sedaily.com/NewsView/1Z1G2AWK84
3742,메트로신문,"최근 자율주행 배달·물류·순찰로봇 등 출시 잇따라, 자율주행 로봇 시장 경쟁 '후끈'",1일 전,시장조사기관 스트래티지 애널리틱스에 따르면 배송(배달)·물류 로봇 등 전문 서비스 ...,http://www.metroseoul.co.kr/article/2020041250...
3743,디지털데일리언론사 선정,"LG폰 브랜드 ‘통일→개성’ 전환…벨벳, 제2의 초콜릿 될까(종합)",1일 전,시장조사기관 스트래티지애널리틱스(SA)에 따르면 작년 LG전자 판매량 순위는 8위다...,http://www.ddaily.co.kr/news/article.html?no=1...
3744,디지털타임스,先극장 後온라인시대 막내리나,1일 전,시장조사업체 스트래티지 애널리틱스는 최근 넷플릭스를 포함한 글로벌 OTT 신규 가입...,http://www.dt.co.kr/contents.html?article_no=2...


In [141]:
df2.to_excel('rival_7.xlsx', index=True)

In [136]:
print(len(press_list))
print(len(title_list))
print(len(date_list))
print(len(content_list))
print(len(href_list))

3746
3746
3746
3746
3746


In [138]:
# date에 오류가 있는지 확인 후 변환하기
df2 = pd.read_excel('rere1.xlsx')
df2

,Unnamed: 0,press,title,date,content,url
0,0,동아일보,갤S10에 7배 빠른 ‘모바일 두뇌’ 탑재,2018.11.15,될 것”이라고 카운터포인트리서치는 지난해 10월 분석했다. 한편 시장조사업체 스트래...,http://news.donga.com/3/all/20181114/92877192/1
1,1,뉴시스언론사 선정,AI스피커 올해 1억대 넘나? 中 알리바바·바이두 '맹추격',2018.11.15,15일 글로벌 시장조사업체 스트래티지 애널리틱스(SA)에 따르면 올해 3분기 글로벌...,http://www.newsis.com/view/?id=NISX20181115_00...
2,2,뉴스토마토,"LG전자 휴대폰, 글로벌 점유율 1%대로 하락",2018.11.15,5%에서 올 3분기 18%로 후퇴했다. 시장조사기관 스트래티지애널리틱스(SA)와 카...,http://www.newstomato.com/ReadNews.aspx?no=858336
3,3,ZDNet Korea,"AI스피커 각축…아마존, 다시 구글 제쳤다",2018.11.16,시장조사업체 스트래티지 애널리틱스는 지난 9월 구글 홈 미니가 2분기 음성 스피커 ...,http://www.zdnet.co.kr/ArticleView.asp?artice_...
4,4,뉴스핌,"'AI·5G' 카드 꺼낸 삼성, 내년 비메모리 지각변화 기대",2018.11.16,시장조사업체 스트래티지애널리틱스(SA)에 따르면 올해 삼성전자와 퀄컴의 모바일 프로...,http://www.newspim.com/news/view/20181116000437
...,...,...,...,...,...,...
3741,3741,서울경제언론사 선정,"삼성전자, 코로나 위기서도 반도체 덕분에 버텼다",1일 전,IM(IT·모바일) 부문 또한 스트래티지애널리틱스(SA)가 올해 글로벌 스마트폰 판...,https://www.sedaily.com/NewsView/1Z1G2AWK84
3742,3742,메트로신문,"최근 자율주행 배달·물류·순찰로봇 등 출시 잇따라, 자율주행 로봇 시장 경쟁 '후끈'",1일 전,시장조사기관 스트래티지 애널리틱스에 따르면 배송(배달)·물류 로봇 등 전문 서비스 ...,http://www.metroseoul.co.kr/article/2020041250...
3743,3743,디지털데일리언론사 선정,"LG폰 브랜드 ‘통일→개성’ 전환…벨벳, 제2의 초콜릿 될까(종합)",1일 전,시장조사기관 스트래티지애널리틱스(SA)에 따르면 작년 LG전자 판매량 순위는 8위다...,http://www.ddaily.co.kr/news/article.html?no=1...
3744,3744,디지털타임스,先극장 後온라인시대 막내리나,1일 전,시장조사업체 스트래티지 애널리틱스는 최근 넷플릭스를 포함한 글로벌 OTT 신규 가입...,http://www.dt.co.kr/contents.html?article_no=2...


In [88]:
print(datetime.now())
time.strftime('%Y-%m-%d', time.localtime(time.time()))

2020-04-14 02:49:59.862528


'2020-04-14'

In [139]:
for i in range(len(df2)):
    print(df2.date[i])
    if "일" in df2.date[i]:
        match = re.search('\d+', df2.date[i])
        now = datetime.now() #현재 시간(수집 시간)
        diff = int(match.group()) #몇일 전/몇시간 전인지
        date = now + timedelta(days=-diff)
        df2.date[i] = date.strftime("%Y.%m.%d")
    
    elif "시간" in df2.date[i]:
        match = re.search('\d+', df2.date[i])
        now = datetime.now() #현재 시간(수집 시간)
        diff = int(match.group()) #몇일 전/몇시간 전인지
        date = now + timedelta(hours=-diff)
        df2.date[i] = date.strftime("%Y.%m.%d") 
    else: pass
    print(i)

2018.11.15
0
2018.11.15
1
2018.11.15
2
2018.11.16
3
2018.11.16
4
2018.11.17
5
2018.11.18
6
2018.11.19
7
2018.11.19
8
2018.11.19
9
2018.11.19
10
2018.11.20
11
2018.11.20
12
2018.11.20
13
2018.11.20
14
2018.11.20
15
2018.11.20
16
2018.11.20
17
2018.11.20
18
2018.11.21
19
2018.11.21
20
2018.11.21
21
2018.11.22
22
2018.11.22
23
2018.11.22
24
2018.11.22
25
2018.11.22
26
2018.11.22
27
2018.11.22
28
2018.11.22
29
2018.11.22
30
2018.11.22
31
2018.11.22
32
2018.11.22
33
2018.11.22
34
2018.11.22
35
2018.11.23
36
2018.11.23
37
2018.11.23
38
2018.11.25
39
2018.11.25
40
2018.11.25
41
2018.11.25
42
2018.11.25
43
2018.11.25
44
2018.11.25
45
2018.11.25
46
2018.11.25
47
2018.11.25
48
2018.11.25
49
2018.11.25
50
2018.11.25
51
2018.11.25
52
2018.11.25
53
2018.11.25
54
2018.11.25
55
2018.11.25
56
2018.11.25
57
2018.11.25
58
2018.11.26
59
2018.11.26
60
2018.11.26
61
2018.11.26
62
2018.11.26
63
2018.11.26
64
2018.11.26
65
2018.11.26
66
2018.11.26
67
2018.11.26
68
2018.11.26
69
2018.11.26
70
2018.11.27
71
20

653
2019.02.06
654
2019.02.06
655
2019.02.06
656
2019.02.06
657
2019.02.06
658
2019.02.07
659
2019.02.07
660
2019.02.07
661
2019.02.07
662
2019.02.07
663
2019.02.08
664
2019.02.08
665
2019.02.08
666
2019.02.08
667
2019.02.08
668
2019.02.08
669
2019.02.10
670
2019.02.10
671
2019.02.10
672
2019.02.10
673
2019.02.10
674
2019.02.10
675
2019.02.10
676
2019.02.10
677
2019.02.10
678
2019.02.10
679
2019.02.10
680
2019.02.10
681
2019.02.10
682
2019.02.10
683
2019.02.10
684
2019.02.10
685
2019.02.10
686
2019.02.10
687
2019.02.10
688
2019.02.10
689
2019.02.11
690
2019.02.11
691
2019.02.11
692
2019.02.11
693
2019.02.11
694
2019.02.11
695
2019.02.11
696
2019.02.11
697
2019.02.11
698
2019.02.11
699
2019.02.12
700
2019.02.12
701
2019.02.12
702
2019.02.12
703
2019.02.13
704
2019.02.13
705
2019.02.13
706
2019.02.13
707
2019.02.13
708
2019.02.13
709
2019.02.13
710
2019.02.13
711
2019.02.13
712
2019.02.13
713
2019.02.14
714
2019.02.14
715
2019.02.14
716
2019.02.14
717
2019.02.14
718
2019.02.14
719
2019.0

2019.06.10
1680
2019.06.10
1681
2019.06.10
1682
2019.06.10
1683
2019.06.10
1684
2019.06.10
1685
2019.06.10
1686
2019.06.11
1687
2019.06.11
1688
2019.06.11
1689
2019.06.11
1690
2019.06.11
1691
2019.06.11
1692
2019.06.11
1693
2019.06.11
1694
2019.06.11
1695
2019.06.11
1696
2019.06.11
1697
2019.06.12
1698
2019.06.12
1699
2019.06.12
1700
2019.06.12
1701
2019.06.12
1702
2019.06.12
1703
2019.06.12
1704
2019.06.12
1705
2019.06.13
1706
2019.06.14
1707
2019.06.14
1708
2019.06.14
1709
2019.06.14
1710
2019.06.14
1711
2019.06.14
1712
2019.06.14
1713
2019.06.14
1714
2019.06.14
1715
2019.06.14
1716
2019.06.14
1717
2019.06.14
1718
2019.06.15
1719
2019.06.16
1720
2019.06.16
1721
2019.06.16
1722
2019.06.16
1723
2019.06.17
1724
2019.06.17
1725
2019.06.17
1726
2019.06.17
1727
2019.06.17
1728
2019.06.17
1729
2019.06.17
1730
2019.06.17
1731
2019.06.17
1732
2019.06.17
1733
2019.06.17
1734
2019.06.17
1735
2019.06.17
1736
2019.06.18
1737
2019.06.18
1738
2019.06.18
1739
2019.06.19
1740
2019.06.19
1741
2019.06.

2019.11.15
2656
2019.11.15
2657
2019.11.15
2658
2019.11.15
2659
2019.11.17
2660
2019.11.17
2661
2019.11.17
2662
2019.11.17
2663
2019.11.18
2664
2019.11.18
2665
2019.11.18
2666
2019.11.18
2667
2019.11.18
2668
2019.11.19
2669
2019.11.19
2670
2019.11.19
2671
2019.11.20
2672
2019.11.20
2673
2019.11.20
2674
2019.11.20
2675
2019.11.21
2676
2019.11.21
2677
2019.11.21
2678
2019.11.21
2679
2019.11.21
2680
2019.11.21
2681
2019.11.21
2682
2019.11.21
2683
2019.11.21
2684
2019.11.21
2685
2019.11.22
2686
2019.11.22
2687
2019.11.22
2688
2019.11.22
2689
2019.11.23
2690
2019.11.23
2691
2019.11.24
2692
2019.11.25
2693
2019.11.25
2694
2019.11.25
2695
2019.11.25
2696
2019.11.25
2697
2019.11.25
2698
2019.11.26
2699
2019.11.26
2700
2019.11.26
2701
2019.11.26
2702
2019.11.26
2703
2019.11.27
2704
2019.11.27
2705
2019.11.27
2706
2019.11.27
2707
2019.11.27
2708
2019.11.27
2709
2019.11.27
2710
2019.11.27
2711
2019.11.27
2712
2019.11.28
2713
2019.11.28
2714
2019.11.28
2715
2019.11.28
2716
2019.11.29
2717
2019.11.

C:\Users\heian\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\heian\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [140]:
df2

,Unnamed: 0,press,title,date,content,url
0,0,동아일보,갤S10에 7배 빠른 ‘모바일 두뇌’ 탑재,2018.11.15,될 것”이라고 카운터포인트리서치는 지난해 10월 분석했다. 한편 시장조사업체 스트래...,http://news.donga.com/3/all/20181114/92877192/1
1,1,뉴시스언론사 선정,AI스피커 올해 1억대 넘나? 中 알리바바·바이두 '맹추격',2018.11.15,15일 글로벌 시장조사업체 스트래티지 애널리틱스(SA)에 따르면 올해 3분기 글로벌...,http://www.newsis.com/view/?id=NISX20181115_00...
2,2,뉴스토마토,"LG전자 휴대폰, 글로벌 점유율 1%대로 하락",2018.11.15,5%에서 올 3분기 18%로 후퇴했다. 시장조사기관 스트래티지애널리틱스(SA)와 카...,http://www.newstomato.com/ReadNews.aspx?no=858336
3,3,ZDNet Korea,"AI스피커 각축…아마존, 다시 구글 제쳤다",2018.11.16,시장조사업체 스트래티지 애널리틱스는 지난 9월 구글 홈 미니가 2분기 음성 스피커 ...,http://www.zdnet.co.kr/ArticleView.asp?artice_...
4,4,뉴스핌,"'AI·5G' 카드 꺼낸 삼성, 내년 비메모리 지각변화 기대",2018.11.16,시장조사업체 스트래티지애널리틱스(SA)에 따르면 올해 삼성전자와 퀄컴의 모바일 프로...,http://www.newspim.com/news/view/20181116000437
...,...,...,...,...,...,...
3741,3741,서울경제언론사 선정,"삼성전자, 코로나 위기서도 반도체 덕분에 버텼다",2020.04.13,IM(IT·모바일) 부문 또한 스트래티지애널리틱스(SA)가 올해 글로벌 스마트폰 판...,https://www.sedaily.com/NewsView/1Z1G2AWK84
3742,3742,메트로신문,"최근 자율주행 배달·물류·순찰로봇 등 출시 잇따라, 자율주행 로봇 시장 경쟁 '후끈'",2020.04.13,시장조사기관 스트래티지 애널리틱스에 따르면 배송(배달)·물류 로봇 등 전문 서비스 ...,http://www.metroseoul.co.kr/article/2020041250...
3743,3743,디지털데일리언론사 선정,"LG폰 브랜드 ‘통일→개성’ 전환…벨벳, 제2의 초콜릿 될까(종합)",2020.04.13,시장조사기관 스트래티지애널리틱스(SA)에 따르면 작년 LG전자 판매량 순위는 8위다...,http://www.ddaily.co.kr/news/article.html?no=1...
3744,3744,디지털타임스,先극장 後온라인시대 막내리나,2020.04.13,시장조사업체 스트래티지 애널리틱스는 최근 넷플릭스를 포함한 글로벌 OTT 신규 가입...,http://www.dt.co.kr/contents.html?article_no=2...
